# 3.4B transformer based on pile uncopyrighted and 40 gig of data

In [1]:
%pip install tiktoken tqdm datasets tiktoken zstandard "fsspec[compression]"

# RESTART THE KERNEL IF RUNNING ON RUNPOD AFTER THIS PIP INSTALL

In [2]:
# -------- MODEL PARAMS --------
n_layers    = 64
n_embd      = 2048
n_heads     = 16
context_len = 1024
batch_size  = 8    # with grad accumulation
dropout     = 0
lr          = 3e-6
from tiktoken import get_encoding
tokenizer = get_encoding("gpt2")
vocab_size  = tokenizer.n_vocab

# ------------------------------

# *cool lr sinusoid trick*

In [3]:
import math

def get_lr(step, max_steps, base_lr=lr, warmup_steps=2000):
    """
    Warmup + cosine decay learning rate schedule.

    Args:
        step (int): current training step
        max_steps (int): total training steps
        base_lr (float): peak learning rate
        warmup_steps (int): number of warmup steps

    Returns:
        float: learning rate for this step
    """
    if step < warmup_steps:
        return base_lr * step / warmup_steps

    progress = (step - warmup_steps) / max(1, max_steps - warmup_steps)
    return base_lr * 0.5 * (1.0 + math.cos(math.pi * progress))


In [4]:
import torch
from torch import nn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
class TokenEmbedding(nn.Module):
    def __init__(self):
        super().__init__()
        self.token = nn.Embedding(vocab_size, n_embd)
        self.pos   = nn.Embedding(context_len, n_embd)

    def forward(self, x):
        B, T = x.shape
        tok = self.token(x)                    # (B, T, C)
        pos = self.pos(torch.arange(T))        # (T, C)
        return tok + pos
emb = TokenEmbedding()



# Above is the definition of our embeddings

# Now define attention

In [6]:
class CausalSelfAttention(nn.Module):
    def __init__(self, n_embd, n_heads, dropout=0.0):
        super().__init__()
        assert n_embd % n_heads == 0

        self.n_heads = n_heads
        self.head_dim = n_embd // n_heads

        self.qkv = nn.Linear(n_embd, 3 * n_embd, bias=False)
        self.proj = nn.Linear(n_embd, n_embd, bias=False)
        self.dropout = dropout

    def forward(self, x):
        B, T, C = x.shape

        qkv = self.qkv(x)                       # (B, T, 3C)
        q, k, v = qkv.chunk(3, dim=-1)

        q = q.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
        k = k.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
        v = v.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)

        # this is the critical line
        y = F.scaled_dot_product_attention(
            q, k, v,
            is_causal=True,
            dropout_p=self.dropout if self.training else 0.0,
        )

        y = y.transpose(1, 2).contiguous().view(B, T, C)
        return self.proj(y)


# Constants again and positional embeddings


In [7]:
class TokenPosEmbedding(nn.Module):
    def __init__(self):
        super().__init__()
        self.token = nn.Embedding(vocab_size, n_embd)
        self.pos   = nn.Embedding(context_len, n_embd)

    def forward(self, x):
        B, T = x.shape
        tok = self.token(x)                                # (B,T,C)
        pos = self.pos(torch.arange(T, device=x.device))  # (T,C)
        return tok + pos


# After attention we have basic FFN

In [9]:
class FeedForward(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.GELU(),
            nn.Linear(4 * n_embd, n_embd),
        )

    def forward(self, x):
        return self.net(x)


In [10]:
class Block(nn.Module):
    def __init__(self):
        super().__init__()
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

        self.attn = CausalSelfAttention(n_embd, n_heads, dropout)
        self.ff   = FeedForward()

    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.ff(self.ln2(x))
        return x


In [11]:
class TransformerLM(nn.Module):
    def __init__(self):
        super().__init__()

        # token + positional embeddings
        self.token_emb = nn.Embedding(vocab_size, n_embd)
        self.pos_emb   = nn.Embedding(context_len, n_embd)

        self.blocks = nn.Sequential(
            *[Block() for _ in range(n_layers)]
        )

        # final normalization + LM head
        self.ln_f = nn.LayerNorm(n_embd)
        self.head = nn.Linear(n_embd, vocab_size)

    def forward(self, x, targets=None):
        B, T = x.shape

        # embeddings
        tok = self.token_emb(x)                              # (B,T,C)
        pos = self.pos_emb(torch.arange(T, device=x.device))# (T,C)
        x = tok + pos                                        # (B,T,C)

        # APPLY ALL BLOCKS HERE
        x = self.blocks(x)

        # final projection
        x = self.ln_f(x)
        logits = self.head(x)                                # (B,T,vocab)

        loss = None
        if targets is not None:
            logits = logits.view(B*T, vocab_size)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss



In [ ]:
model = TransformerLM().to(device)


In [ ]:
print(sum(p.numel() for p in model.parameters()) / 1e6, "M params")


# optimus

In [ ]:
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=lr
)


In [ ]:
@torch.no_grad()
def estimate_loss():
    model.eval()
    out = {}
    for split in ["train", "test"]:
        losses = []
        for _ in range(20):
            xb, yb = get_batch(split)
            xb, yb = xb.to(device), yb.to(device)
            _, loss = model(xb, yb)
            losses.append(loss.item())
        out[split] = sum(losses) / len(losses)
    model.train() # go back because we will be in a train loop
    return out


In [ ]:
def token_stream(hf_dataset, tokenizer):
    """
    Lazily yields lists of token ids from a streaming HF dataset.
    """
    for ex in hf_dataset:
        text = ex.get("text", "")
        if not text:
            continue
        ids = tokenizer.encode(text)
        if len(ids) > 1:
            yield ids
import random
import torch

def window_stream(token_iter, context_len, device):
    """
    Yields single (x, y) training samples of shape [context_len].
    """
    buffer = []

    for ids in token_iter:
        buffer.extend(ids)

        while len(buffer) >= context_len + 1:
            start = random.randint(0, len(buffer) - context_len - 1)

            x = buffer[start : start + context_len]
            y = buffer[start + 1 : start + context_len + 1]

            yield (
                torch.tensor(x, dtype=torch.long, device=device),
                torch.tensor(y, dtype=torch.long, device=device),
            )
def batch_stream(sample_iter, batch_size):
    """
    Groups single samples into batches.
    """
    while True:
        xb, yb = zip(*(next(sample_iter) for _ in range(batch_size)))
        yield torch.stack(xb), torch.stack(yb)
from datasets import load_dataset

# load streaming dataset
ds = load_dataset(
    "monology/pile-uncopyrighted",
    split="train",
    streaming=True,
)

# tokenizer
tokenizer = get_encoding("gpt2")

# build streams
tok_iter   = token_stream(ds, tokenizer)
sample_iter = window_stream(tok_iter, context_len, device)
batch_iter  = batch_stream(sample_iter, batch_size)


In [ ]:
from tqdm.notebook import tqdm
import torch
import os
import torch.nn.functional as F

# ---- checkpoint setup ----
CKPT_DIR = "checkpoints"
os.makedirs(CKPT_DIR, exist_ok=True)

scaler = torch.cuda.amp.GradScaler()

max_steps = 9000
history = []

model.train()

for step in tqdm(range(max_steps)):
    # ---- LR schedule ----
    lr = get_lr(step, max_steps)
    for g in optimizer.param_groups:
        g["lr"] = lr

    # ---- get batch (CPU tensors) ----
    xb, yb = next(batch_iter)

    # move to GPU ONCE per batch
    xb = xb.to(device, non_blocking=True)
    yb = yb.to(device, non_blocking=True)

    # ---- forward + backward (AMP) ----
    with torch.cuda.amp.autocast():
        logits, loss = model(xb, yb)

    optimizer.zero_grad(set_to_none=True)
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()

    # ---- logging ----
    loss_val = loss.item()
    history.append(loss_val)

    if step % 10 == 0:
        print(f"step {step:5d} | loss {loss_val:.4f}")

    # ---- checkpoint every 300 steps ----
    if step > 0 and step % 300 == 0:
        ckpt_path = os.path.join(CKPT_DIR, f"ckpt_step_{step}.pt")
        torch.save(
            {
                "model": model.state_dict(),
                "optimizer": optimizer.state_dict(),
                "scaler": scaler.state_dict(),
                "step": step,
            },
            ckpt_path,
        )
        print(f"[checkpoint saved → {ckpt_path}]")


In [ ]:
print(xb[0][:10])
print(yb[0][:10])


In [ ]:
import torch

@torch.no_grad()
def complete(
    model,
    tokenizer,
    prompt: str,
    max_new_tokens: int = 50,
    temperature: float = 1.0,
    top_k: int | None = 50,
    device: str = "cuda",
):
    model.eval()

    # encode prompt
    idx = torch.tensor(
        [tokenizer.encode(prompt)],
        dtype=torch.long,
        device=device,
    )

    for _ in range(max_new_tokens):
        # crop context if needed
        idx_cond = idx[:, -context_len :]

        logits, _ = model(idx_cond)
        logits = logits[:, -1, :] / temperature

        if top_k is not None:
            v, _ = torch.topk(logits, top_k)
            logits[logits < v[:, [-1]]] = -float("inf")

        probs = F.softmax(logits, dim=-1)
        next_token = torch.multinomial(probs, num_samples=1)

        idx = torch.cat([idx, next_token], dim=1)

    # decode only the completion
    completion = tokenizer.decode(idx[0].tolist())
    return completion


In [ ]:
from tiktoken import get_encoding


text = complete(
    model,
    tokenizer,
    prompt="Hi,",
    max_new_tokens=60,
    temperature=0.2,
    top_k=40,
    device=device,
)

print(text)


In [ ]:
torch.cuda.empty_cache()